# Phase 7: IFC Export (Semantic Property Sets)
## Alaca Cesmesi Scan-to-HBIM V6 Pipeline

Creates IFC4 files with:
- IfcTriangulatedFaceSet geometry
- 3 Semantic Property Sets (Pset_YapiKimligi, Pset_ElemanBilgisi, Pset_KorumaDurumu)
- Proper IFC classes per element

**Output IFC Classes:**
- zemin -> IfcSlab
- seki -> IfcSlab
- ana_cephe -> IfcWall
- kemer -> IfcBuildingElementProxy
- sacak -> IfcRoof

**Input:** `gs://alaca-cesme-hbim-v6/processed/v{N}/06_mesh/`  
**Output:** `gs://alaca-cesme-hbim-v6/processed/v{N}/07_ifc/`

In [ ]:
!pip install -q open3d google-cloud-storage numpy ifcopenshell

In [ ]:
import open3d as o3d
import numpy as np
import ifcopenshell
import ifcopenshell.guid
import json
import os
import time
from datetime import datetime
from google.cloud import storage
from google.colab import auth

auth.authenticate_user()

# Configuration
BUCKET_NAME = "alaca-cesme-hbim-v6"
PROJECT_ID = "concrete-racer-470219-h8"
VERSION = "v1"

# Element configurations
ELEMENTS = {
    "zemin": {"ifc_class": "IfcSlab", "name_tr": "Zemin", "name_en": "Ground"},
    "seki": {"ifc_class": "IfcSlab", "name_tr": "Seki", "name_en": "Platform"},
    "ana_cephe": {"ifc_class": "IfcWall", "name_tr": "Ana Duvar", "name_en": "Main Wall"},
    "kemer": {"ifc_class": "IfcBuildingElementProxy", "name_tr": "Kemer", "name_en": "Arch"},
    "sacak": {"ifc_class": "IfcRoof", "name_tr": "Sacak", "name_en": "Cornice"}
}

# Heritage metadata
HERITAGE_METADATA = {
    "identity": {
        "structure_name": "Alaca Cesmesi",
        "construction_date": "1586",
        "hijri_date": "H.995",
        "period": "Classical Ottoman (III. Murad)",
        "patron": "Suleyman Aga",
        "structure_type": "Cesme (Fountain)"
    },
    "location": {
        "province": "Istanbul",
        "district": "Eyupsultan",
        "neighborhood": "Eyup Sultan Mahallesi",
        "coordinates": "41.0478, 28.9342"
    },
    "conservation": {
        "registration_status": "Registered Cultural Heritage",
        "last_restoration": "2014",
        "current_condition": "Restored",
        "original_function": "Public Fountain",
        "current_function": "Historical Monument"
    }
}

# Paths
INPUT_BASE = f"processed/{VERSION}/06_mesh/"
OUTPUT_BASE = f"processed/{VERSION}/07_ifc/"

In [ ]:
# GCS functions
def download_from_gcs(bucket_name, blob_name, local_path):
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(local_path)
    print(f"Downloaded: {blob_name}")
    return local_path

def upload_to_gcs(bucket_name, local_path, blob_name):
    client = storage.Client(project=PROJECT_ID)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(local_path)
    print(f"Uploaded: {blob_name}")
    return f"gs://{bucket_name}/{blob_name}"

In [ ]:
def create_ifc_with_semantic_properties(mesh, element_key, element_config, metadata):
    """
    Create IFC4 file with semantic property sets.
    
    Property Sets:
    1. Pset_YapiKimligi: Building identity
    2. Pset_ElemanBilgisi: Element information
    3. Pset_KorumaDurumu: Conservation status
    """
    ifc = ifcopenshell.file(schema="IFC4")
    
    # Project
    project = ifc.create_entity("IfcProject", ifcopenshell.guid.new())
    project.Name = "Alaca Cesmesi HBIM V6"
    
    # Units
    length_unit = ifc.create_entity("IfcSIUnit", UnitType="LENGTHUNIT", Name="METRE")
    unit_assignment = ifc.create_entity("IfcUnitAssignment", Units=[length_unit])
    project.UnitsInContext = unit_assignment
    
    # Context
    context = ifc.create_entity("IfcGeometricRepresentationContext",
        ContextType="Model", CoordinateSpaceDimension=3, Precision=1e-5)
    project.RepresentationContexts = [context]
    
    # Hierarchy
    site = ifc.create_entity("IfcSite", ifcopenshell.guid.new(), Name="Eyupsultan")
    building = ifc.create_entity("IfcBuilding", ifcopenshell.guid.new(),
        Name=metadata["identity"]["structure_name"])
    storey = ifc.create_entity("IfcBuildingStorey", ifcopenshell.guid.new(), Name="Zemin Kat")
    
    ifc.create_entity("IfcRelAggregates", ifcopenshell.guid.new(),
        RelatingObject=project, RelatedObjects=[site])
    ifc.create_entity("IfcRelAggregates", ifcopenshell.guid.new(),
        RelatingObject=site, RelatedObjects=[building])
    ifc.create_entity("IfcRelAggregates", ifcopenshell.guid.new(),
        RelatingObject=building, RelatedObjects=[storey])
    
    # Mesh geometry
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    
    coord_list = [[float(v[0]), float(v[1]), float(v[2])] for v in vertices]
    coords = ifc.create_entity("IfcCartesianPointList3D", CoordList=coord_list)
    
    indices = [[int(t[0]+1), int(t[1]+1), int(t[2]+1)] for t in triangles]
    face_set = ifc.create_entity("IfcTriangulatedFaceSet",
        Coordinates=coords, CoordIndex=indices, Closed=False)
    
    # Shape representation
    shape_rep = ifc.create_entity("IfcShapeRepresentation",
        ContextOfItems=context, RepresentationIdentifier="Body",
        RepresentationType="Tessellation", Items=[face_set])
    product_rep = ifc.create_entity("IfcProductDefinitionShape", Representations=[shape_rep])
    
    # Placement
    origin = ifc.create_entity("IfcCartesianPoint", Coordinates=(0.0, 0.0, 0.0))
    placement = ifc.create_entity("IfcLocalPlacement",
        RelativePlacement=ifc.create_entity("IfcAxis2Placement3D", Location=origin))
    
    # Create element
    element = ifc.create_entity(element_config["ifc_class"], ifcopenshell.guid.new())
    element.Name = element_config["name_tr"]
    element.Description = element_config["name_en"]
    element.ObjectPlacement = placement
    element.Representation = product_rep
    
    ifc.create_entity("IfcRelContainedInSpatialStructure", ifcopenshell.guid.new(),
        RelatingStructure=storey, RelatedElements=[element])
    
    # ============================================
    # PROPERTY SET 1: Pset_YapiKimligi
    # ============================================
    props1 = [
        ifc.create_entity("IfcPropertySingleValue", Name="YapiAdi",
            NominalValue=ifc.create_entity("IfcText", metadata["identity"]["structure_name"])),
        ifc.create_entity("IfcPropertySingleValue", Name="YapimTarihi",
            NominalValue=ifc.create_entity("IfcText", metadata["identity"]["construction_date"])),
        ifc.create_entity("IfcPropertySingleValue", Name="HicriTarih",
            NominalValue=ifc.create_entity("IfcText", metadata["identity"]["hijri_date"])),
        ifc.create_entity("IfcPropertySingleValue", Name="Donem",
            NominalValue=ifc.create_entity("IfcText", metadata["identity"]["period"])),
        ifc.create_entity("IfcPropertySingleValue", Name="Bani",
            NominalValue=ifc.create_entity("IfcText", metadata["identity"]["patron"])),
        ifc.create_entity("IfcPropertySingleValue", Name="YapiTipi",
            NominalValue=ifc.create_entity("IfcText", metadata["identity"]["structure_type"])),
        ifc.create_entity("IfcPropertySingleValue", Name="Konum",
            NominalValue=ifc.create_entity("IfcText", f"{metadata['location']['district']}, {metadata['location']['province']}")),
    ]
    pset1 = ifc.create_entity("IfcPropertySet", ifcopenshell.guid.new(),
        Name="Pset_YapiKimligi", HasProperties=props1)
    ifc.create_entity("IfcRelDefinesByProperties", ifcopenshell.guid.new(),
        RelatingPropertyDefinition=pset1, RelatedObjects=[element])
    
    # ============================================
    # PROPERTY SET 2: Pset_ElemanBilgisi
    # ============================================
    props2 = [
        ifc.create_entity("IfcPropertySingleValue", Name="ElemanAdi_TR",
            NominalValue=ifc.create_entity("IfcText", element_config["name_tr"])),
        ifc.create_entity("IfcPropertySingleValue", Name="ElemanAdi_EN",
            NominalValue=ifc.create_entity("IfcText", element_config["name_en"])),
        ifc.create_entity("IfcPropertySingleValue", Name="IFCSinifi",
            NominalValue=ifc.create_entity("IfcText", element_config["ifc_class"])),
        ifc.create_entity("IfcPropertySingleValue", Name="VertexSayisi",
            NominalValue=ifc.create_entity("IfcInteger", len(vertices))),
        ifc.create_entity("IfcPropertySingleValue", Name="UcgenSayisi",
            NominalValue=ifc.create_entity("IfcInteger", len(triangles))),
    ]
    pset2 = ifc.create_entity("IfcPropertySet", ifcopenshell.guid.new(),
        Name="Pset_ElemanBilgisi", HasProperties=props2)
    ifc.create_entity("IfcRelDefinesByProperties", ifcopenshell.guid.new(),
        RelatingPropertyDefinition=pset2, RelatedObjects=[element])
    
    # ============================================
    # PROPERTY SET 3: Pset_KorumaDurumu
    # ============================================
    props3 = [
        ifc.create_entity("IfcPropertySingleValue", Name="TescilDurumu",
            NominalValue=ifc.create_entity("IfcText", metadata["conservation"]["registration_status"])),
        ifc.create_entity("IfcPropertySingleValue", Name="SonRestorasyon",
            NominalValue=ifc.create_entity("IfcText", metadata["conservation"]["last_restoration"])),
        ifc.create_entity("IfcPropertySingleValue", Name="MevcutDurum",
            NominalValue=ifc.create_entity("IfcText", metadata["conservation"]["current_condition"])),
        ifc.create_entity("IfcPropertySingleValue", Name="OzgunIslev",
            NominalValue=ifc.create_entity("IfcText", metadata["conservation"]["original_function"])),
        ifc.create_entity("IfcPropertySingleValue", Name="GuncelIslev",
            NominalValue=ifc.create_entity("IfcText", metadata["conservation"]["current_function"])),
    ]
    pset3 = ifc.create_entity("IfcPropertySet", ifcopenshell.guid.new(),
        Name="Pset_KorumaDurumu", HasProperties=props3)
    ifc.create_entity("IfcRelDefinesByProperties", ifcopenshell.guid.new(),
        RelatingPropertyDefinition=pset3, RelatedObjects=[element])
    
    return ifc

In [ ]:
start_time = time.time()
ifc_stats = {}

for element_key, element_config in ELEMENTS.items():
    print(f"\n{'='*60}")
    print(f"Creating IFC: {element_key.upper()}")
    print(f"{'='*60}")
    
    # Download mesh
    local_mesh = f"/content/{element_key}_mesh.ply"
    try:
        download_from_gcs(BUCKET_NAME, f"{INPUT_BASE}{element_key}_poisson.ply", local_mesh)
    except Exception as e:
        print(f"  Skipping {element_key}: {e}")
        continue
    
    mesh = o3d.io.read_triangle_mesh(local_mesh)
    print(f"  Loaded mesh: {len(mesh.vertices):,} vertices, {len(mesh.triangles):,} triangles")
    
    # Create IFC
    print(f"  Creating IFC ({element_config['ifc_class']})...")
    ifc_file = create_ifc_with_semantic_properties(mesh, element_key, element_config, HERITAGE_METADATA)
    
    # Save
    local_ifc = f"/content/alaca_cesme_{element_key}.ifc"
    ifc_file.write(local_ifc)
    size_mb = os.path.getsize(local_ifc) / (1024 * 1024)
    print(f"  Saved: {local_ifc} ({size_mb:.1f} MB)")
    
    upload_to_gcs(BUCKET_NAME, local_ifc, f"{OUTPUT_BASE}alaca_cesme_{element_key}.ifc")
    
    ifc_stats[element_key] = {
        "ifc_class": element_config["ifc_class"],
        "vertices": len(mesh.vertices),
        "triangles": len(mesh.triangles),
        "size_mb": round(size_mb, 1),
        "property_sets": 3
    }

elapsed_time = time.time() - start_time
print(f"\nTotal processing time: {elapsed_time:.1f} seconds")

In [ ]:
# Summary
print("\n" + "="*60)
print("IFC EXPORT SUMMARY")
print("="*60)
print(f"\n{'Element':<12} {'IFC Class':<25} {'Triangles':>10} {'Size':>8}")
print("-"*60)
for name, stats in ifc_stats.items():
    print(f"{name:<12} {stats['ifc_class']:<25} {stats['triangles']:>10,} {stats['size_mb']:>6.1f} MB")

total_size = sum(s['size_mb'] for s in ifc_stats.values())
print("-"*60)
print(f"{'TOTAL':<12} {'':<25} {sum(s['triangles'] for s in ifc_stats.values()):>10,} {total_size:>6.1f} MB")

# Save stats
final_stats = {
    "phase": "07_ifc",
    "schema": "IFC4",
    "geometry_type": "IfcTriangulatedFaceSet",
    "property_sets": ["Pset_YapiKimligi", "Pset_ElemanBilgisi", "Pset_KorumaDurumu"],
    "elements": ifc_stats,
    "heritage_metadata": HERITAGE_METADATA,
    "processing_time_sec": elapsed_time,
    "timestamp": datetime.now().isoformat(),
    "pipeline_version": "v6"
}

local_stats = "/content/07_ifc_stats.json"
with open(local_stats, 'w') as f:
    json.dump(final_stats, f, indent=2)
upload_to_gcs(BUCKET_NAME, local_stats, f"{OUTPUT_BASE}07_ifc_stats.json")

In [ ]:
# Status for n8n
status = {
    "phase": "07_ifc",
    "status": "success",
    "version": VERSION,
    "outputs": {
        "ifc_files": [f"gs://{BUCKET_NAME}/{OUTPUT_BASE}alaca_cesme_{name}.ifc" for name in ifc_stats.keys()],
        "stats": f"gs://{BUCKET_NAME}/{OUTPUT_BASE}07_ifc_stats.json"
    },
    "metrics": {
        "n_elements": len(ifc_stats),
        "total_size_mb": round(total_size, 1),
        "property_sets": 3,
        "processing_time": f"{elapsed_time:.1f}s"
    },
    "timestamp": datetime.now().isoformat(),
    "pipeline_complete": True,
    "next_step": "Download IFC files and open in Revit (MANUAL)"
}

print("\n" + "="*60)
print("PHASE 7 COMPLETE - PIPELINE FINISHED!")
print("="*60)
print(json.dumps(status, indent=2))
print("\n" + "="*60)
print("NEXT STEP: Download IFC files and open in Revit")
print("="*60)